In [45]:
###############################################################################
##
## Name: 02_data_aggregation.ipynb
## Purpose: 
## Author: VDR
## Email: dellarocca.vale@gmail.com
## Date: 2021-12-05
## Version: v1
##
###############################################################################
##
## Notes: Same output of what provided in April, just rewritten in Python
##
###############################################################################



## 0. General set up ----------------------------------------------------------



import os
import pandas as pd
import numpy as np
import re
import requests
import datetime
from data_quality_checks_vdr import data_quality_checks_f
import matplotlib.pyplot as plt
from urllib.request import urlopen, urlretrieve
from functools import reduce

os.getcwd()

cols_to_keep = ["as_of_date", "delayable", "social", "staple", "work_related"]
cols_to_keep_ea = ["as_of_date", "a_delayable", "a_social", "a_staple", "a_work_related"]



## 1. Read the files downloaded with the previous code --------------------------------------------------------------



dataset_cs = pd.read_excel("raw_data/card_spending_021221.xlsx", sheet_name = 1, index_col = None, skiprows = 3, dtype = str).head(6)
dataset_i = pd.read_excel("raw_data/ios_data_ios1.xlsx", sheet_name = 0, index_col = None, dtype = str).iloc[6:]
dataset_f = pd.read_excel("raw_data/furlough_data.xlsx", sheet_name = "3. Time series by sector", header = 3, dtype = str)[:-5]
dataset_e = pd.read_excel("raw_data/employment_data.xls", sheet_name = 1, header = 5, dtype = str).iloc[11:][:-16]
dataset_v = pd.read_excel("raw_data/vacancies_data.xls", sheet_name = 1, header = 3, dtype = str).iloc[4:][:-9]
dataset_e_additional = pd.read_excel("raw_data/employment_additional.xls", sheet_name = 1, header = 5, dtype = str).iloc[3:][:-18]


## 2. Card spending cleaning and quality check ------------------------------------------------------

dataset_cs_clean = np.transpose(dataset_cs)


## We will first clean the column names, given the inconsistent format
new_header = dataset_cs_clean.iloc[0]
dataset_cs_clean = dataset_cs_clean[1:]
dataset_cs_clean.columns = new_header
dataset_cs_clean.columns = [c.replace(' ', '_') for c in dataset_cs_clean.columns]
dataset_cs_clean.columns = [x.lower() for x in dataset_cs_clean.columns]


## We will now create new variables as % change compared to the previous reporting date
dataset_cs_clean["lag_delayable"] = dataset_cs_clean.delayable.shift()
dataset_cs_clean["lag_social"] = dataset_cs_clean.social.shift()
dataset_cs_clean["lag_staple"] = dataset_cs_clean.staple.shift()
dataset_cs_clean["lag_work_related"] = dataset_cs_clean.work_related.shift()


dataset_cs_clean["delayable"] = (dataset_cs_clean.delayable.astype(float) - dataset_cs_clean.lag_delayable.astype(float))/dataset_cs_clean.lag_delayable.astype(float)
dataset_cs_clean["social"] = (dataset_cs_clean.social.astype(float) - dataset_cs_clean.lag_social.astype(float))/dataset_cs_clean.lag_social.astype(float)
dataset_cs_clean["staple"] = (dataset_cs_clean.staple.astype(float) - dataset_cs_clean.lag_staple.astype(float))/dataset_cs_clean.lag_staple.astype(float)
dataset_cs_clean["work_related"] = (dataset_cs_clean.work_related.astype(float) - dataset_cs_clean.lag_work_related.astype(float))/dataset_cs_clean.lag_work_related.astype(float)


## Formatting date
dataset_cs_clean["as_of_date"] = dataset_cs_clean["category"]
dataset_cs_clean["as_of_date"] = pd.to_datetime(dataset_cs_clean["as_of_date"])

## Final dataset
dataset_cs_clean = dataset_cs_clean[cols_to_keep]
dataset_cs_clean.name = "dataset_cs_clean"


## Instead of rewriting the code to perform some data quality checks (outlier, missing rate, zero rate,
## and to creates a csv with min/max and some other basic info)
## multiple times for each dataset, we have defined a function to do it 
data_quality_checks_f(dataset_cs_clean)



## 3. IoS cleaning ------------------------------------------------------


dataset_i_clean = dataset_i.copy()

## We will first clean the column names, given the inconsistent format
dataset_i_clean.columns = dataset_i_clean.columns.str.replace(r'\d+', '')
dataset_i_clean.columns = dataset_i_clean.columns.str.replace(': Index-1dp', "")
dataset_i_clean.columns = dataset_i_clean.columns.str.replace(': Index-dp', "")
dataset_i_clean.columns = dataset_i_clean.columns.str.replace('IoS: ', "")
dataset_i_clean.columns = dataset_i_clean.columns.str.replace("&", "")
dataset_i_clean.columns = dataset_i_clean.columns.str.replace(': ', "")
dataset_i_clean.columns = dataset_i_clean.columns.str.replace('#','')
dataset_i_clean.columns = dataset_i_clean.columns.str.replace(",",'')
dataset_i_clean.columns = dataset_i_clean.columns.str.replace("-",'')
dataset_i_clean.columns = [c.replace(' ', '') for c in dataset_i_clean.columns]
dataset_i_clean.columns = [x.lower() for x in dataset_i_clean.columns]


## We will now create the variables we need and cast them in the appropriate format
dataset_i_clean["as_of_date"] = dataset_i_clean.title
dataset_i_clean["delayable"] = dataset_i_clean.wholesaleretailtrade.astype(float)
dataset_i_clean["social"] = dataset_i_clean.gjdistributiontransportaccommodationfoodservicesandinformationcommunication.astype(float)
dataset_i_clean["staple"] = dataset_i_clean.humanhealthactivities.astype(float)
dataset_i_clean["work_related"] = dataset_i_clean.htransportationandstorage.astype(float)
dataset_i_clean = dataset_i_clean[~(dataset_i_clean.as_of_date.str.len() < 8)]
dataset_i_clean["year"] = dataset_i_clean.as_of_date.str[0:4]
dataset_i_clean["year"] = pd.to_numeric(dataset_i_clean.year)
dataset_i_clean = dataset_i_clean[~(dataset_i_clean.year < 2020)]
dataset_i_clean['as_of_date'] = pd.to_datetime(dataset_i_clean['as_of_date'].str[0:5]  + dataset_i_clean['as_of_date'].str[5:6] +  dataset_i_clean['as_of_date'].str[6:8].str.lower(), format='%Y %b')

## Final dataset
dataset_i_clean = dataset_i_clean[cols_to_keep]
dataset_i_clean.name = "dataset_i_clean"


## Data quality checks with the usual function
data_quality_checks_f(dataset_i_clean)



## 4. Furlough cleaning ------------------------------------------------------



dataset_f_clean = dataset_f.copy()

## We will first clean the column names, given the inconsistent format
dataset_f_clean.columns = [x.lower() for x in dataset_f_clean.columns]
dataset_f_clean.columns = dataset_f_clean.columns.str.replace(";","")
dataset_f_clean.columns = dataset_f_clean.columns.str.replace(" ",'_')


## We will now create the variables we need as % change from a reference date (Oct-Dec 2019, so pre-Covid) 
## and cast them in the appropriate format
dataset_f_clean["as_of_date"] = dataset_f_clean.date
dataset_f_clean["as_of_date"] = pd.to_datetime(dataset_f_clean['as_of_date'])
dataset_f_clean["f_delayable"] = dataset_f_clean.wholesale_and_retail_repair_of_motor_vehicles
dataset_f_clean["f_social"] = dataset_f_clean.accommodation_and_food_services
dataset_f_clean["f_staple"] = dataset_f_clean.health_and_social_work
dataset_f_clean["f_work_related"] = dataset_f_clean.transportation_and_storage
dataset_e_additional_clean = dataset_e_additional.copy()
dataset_e_additional_clean["as_of_date"] = dataset_e_additional_clean["Unnamed: 0"]
dataset_e_additional_clean = dataset_e_additional_clean.loc[dataset_e_additional_clean['as_of_date'] == 'Oct-Dec 2019']
dataset_e_additional_clean['as_of_date'] = pd.to_datetime(dataset_e_additional_clean['as_of_date'].str[8:12] + dataset_e_additional_clean['as_of_date'].str[4:7], format='%Y%b')
dataset_e_additional_clean["a_delayable"] = dataset_e_additional_clean["Wholesale, retail & repair of motor vehicles"]
dataset_e_additional_clean["a_social"] = dataset_e_additional_clean["Accommod-ation & food services"]
dataset_e_additional_clean["a_staple"] = dataset_e_additional_clean["Human health & social work activities"]
dataset_e_additional_clean["a_work_related"] = dataset_e_additional_clean["Transport & storage"]
dataset_e_additional_clean = dataset_e_additional_clean[cols_to_keep_ea]
dataset_f_clean["a_delayable"] = int(dataset_e_additional_clean["a_delayable"])
dataset_f_clean["a_social"] = int(dataset_e_additional_clean["a_social"])
dataset_f_clean["a_staple"] = int(dataset_e_additional_clean["a_staple"])
dataset_f_clean["a_work_related"] = int(dataset_e_additional_clean["a_work_related"])
dataset_f_clean["delayable"] = dataset_f_clean.f_delayable.astype(float)/dataset_f_clean.a_delayable.astype(float)
dataset_f_clean["social"] = dataset_f_clean.f_social.astype(float)/dataset_f_clean.a_social.astype(float)
dataset_f_clean["staple"] = dataset_f_clean.f_staple.astype(float)/dataset_f_clean.a_staple.astype(float)
dataset_f_clean["work_related"] = dataset_f_clean.f_work_related.astype(float)/dataset_f_clean.a_work_related.astype(float)

## Final dataset
dataset_f_clean = dataset_f_clean[cols_to_keep]
dataset_f_clean.name = "dataset_f_clean"

## Data quality checks with the usual function
data_quality_checks_f(dataset_f_clean)



## 5. Employment cleaning ------------------------------------------------------


dataset_e_clean = dataset_e.copy()

## We will first clean the column names, given the inconsistent format
dataset_e_clean["as_of_date"] = dataset_e_clean["Unnamed: 0"]


## We will now create the variables we need and cast them in the appropriate format
dataset_e_clean["delayable"] = dataset_e_clean["Wholesale, retail & repair of motor vehicles"].astype(float)
dataset_e_clean["social"] = dataset_e_clean["Accommod-ation & food services"].astype(float)
dataset_e_clean["staple"] = dataset_e_clean["Human health & social work activities"].astype(float)
dataset_e_clean["work_related"] = dataset_e_clean["Transport & storage"].astype(float)
dataset_e_clean['as_of_date'] = pd.to_datetime(dataset_e_clean['as_of_date'].str[8:12] + dataset_e_clean['as_of_date'].str[4:7], format='%Y%b')

## Final dataset
dataset_e_clean = dataset_e_clean[cols_to_keep]
dataset_e_clean.name = "dataset_e_clean"

## Reshape of additional emplyment data
dataset_e_additional_clean_2 = pd.melt(dataset_e_additional_clean, id_vars = "as_of_date")
dataset_e_additional_clean_2 = dataset_e_additional_clean_2.rename(columns={'value': 'average_pre_cv'})

## Data quality checks with the usual function
data_quality_checks_f(dataset_e_clean)


## 5. Vacancies cleaning ------------------------------------------------------


dataset_v_clean = dataset_v.copy()

## We will first clean the column names, given the inconsistent format
dataset_v_clean["as_of_date"] = dataset_v_clean["SIC 2007 sections"]

## We will now create the variables we need and cast them in the appropriate format
dataset_v_clean["delayable"] = dataset_v_clean["Wholesale & retail trade; repair of motor vehicles and motor cycles"].astype(float)
dataset_v_clean["social"] = dataset_v_clean["Accomoda-tion & food service activities"].astype(float)
dataset_v_clean["staple"] = dataset_v_clean["Human health & social work activities"].astype(float)
dataset_v_clean["work_related"] = dataset_v_clean["Transport & storage"].astype(float)
dataset_v_clean["year"] = dataset_v_clean.as_of_date.str[8:12]
dataset_v_clean["year"] = pd.to_numeric(dataset_v_clean.year)
dataset_v_clean = dataset_v_clean[~(dataset_v_clean.year < 2020)]
dataset_v_clean['as_of_date'] = pd.to_datetime(dataset_v_clean['as_of_date'].str[8:12] + dataset_v_clean['as_of_date'].str[4:7], format='%Y%b')

## Final dataset
dataset_v_clean = dataset_v_clean[cols_to_keep]
dataset_v_clean.name = "dataset_v_clean"

## Data quality checks with the usual function
data_quality_checks_f(dataset_v_clean)



## 6. Join the data and check final sample -----------------------------------


## First, we create a range of dates to use as common joining list
## given that the datasets span different time periods
list_name = pd.date_range("2020-03-04","2021-03-31",freq='d').tolist()
dates_list = pd.DataFrame (list_name, columns = ['as_of_date'])
dates_list["as_of_date"] = pd.to_datetime(dates_list['as_of_date'])

## Checking if some dates are completely missing 
## (so NOT if a column has missings values in it, despite having a date asociated with it - which is taken care of by the funcion  "00_data_quality_checks" -
## but to see if entire dates are completely missing)
test_cs = dates_list.merge(dataset_cs_clean, on = "as_of_date")
test_cs = test_cs[test_cs['as_of_date'].dt.dayofweek < 5] ## Only 8 non-weekend dates missing, corresponding to bank holidays


test_f = dates_list.merge(dataset_f_clean, on = "as_of_date")
test_f = test_f.staple.notna() ## The missing values in _f dataset are March 2020 and March 2021, due to the fact that the dataset itself starts and ends after/before that


## Filling in the missing values
additional_cleaning_cs = dates_list.merge(dataset_cs_clean, on = "as_of_date")
additional_cleaning_cs = additional_cleaning_cs.fillna(method='ffill')


## Reshaping the datasets to allow for a clean output
dataset_cs_clean = pd.melt(dataset_cs_clean, id_vars = "as_of_date")
dataset_cs_clean = dataset_cs_clean.rename(columns={'value': 'c_value'})
dataset_i_clean = pd.melt(dataset_i_clean, id_vars = "as_of_date")
dataset_i_clean = dataset_i_clean.rename(columns={'value': 'i_value'})
dataset_f_clean = pd.melt(dataset_f_clean, id_vars = "as_of_date")
dataset_f_clean = dataset_f_clean.rename(columns={'value': 'f_value'})
dataset_e_clean = pd.melt(dataset_e_clean, id_vars = "as_of_date")
dataset_e_clean = dataset_e_clean.rename(columns={'value': 'e_value'})
dataset_v_clean = pd.melt(dataset_v_clean, id_vars = "as_of_date")
dataset_v_clean = dataset_v_clean.rename(columns={'value': 'v_value'})
additional_cleaning_cs = pd.melt(additional_cleaning_cs, id_vars = "as_of_date")
additional_cleaning_cs = additional_cleaning_cs.rename(columns={'value': 'c_value'})

## Final dataset
final_daily_dataset = reduce(lambda left,right: pd.merge(left,right,on=['as_of_date', "variable"], how='left'), [additional_cleaning_cs, dataset_i_clean, dataset_f_clean, dataset_e_clean, dataset_v_clean])
final_daily_dataset.to_csv("final_daily_dataset.csv") 


/Users/Valeria/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Missing rate
as_of_date      0.000000
delayable       0.210084
social          0.210084
staple          0.210084
work_related    0.210084
dtype: float64
Zero rate
as_of_date
0.0
delayable
0.002105263157894737
social
0.0
staple
0.0
work_related
0.004210526315789474


/var/folders/yx/8zh8_jv54k5___mhhnhk40580000gn/T/ipykernel_34523/826538718.py:100: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset_i_clean.columns = dataset_i_clean.columns.str.replace(r'\d+', '')


Missing rate
as_of_date      0.0
delayable       0.0
social          0.0
staple          0.0
work_related    0.0
dtype: float64
Zero rate
as_of_date
0.0
delayable
0.0
social
0.0
staple
0.0
work_related
0.0
Missing rate
as_of_date      0.0
delayable       0.0
social          0.0
staple          0.0
work_related    0.0
dtype: float64
Zero rate
as_of_date
0.0
delayable
0.0
social
0.0
staple
0.0
work_related
0.0
Missing rate
as_of_date      0.0
delayable       0.0
social          0.0
staple          0.0
work_related    0.0
dtype: float64
Zero rate
as_of_date
0.0
delayable
0.0
social
0.0
staple
0.0
work_related
0.0
Missing rate
as_of_date      0.0
delayable       0.0
social          0.0
staple          0.0
work_related    0.0
dtype: float64
Zero rate
as_of_date
0.0
delayable
0.0
social
0.0
staple
0.0
work_related
0.0


<Figure size 432x288 with 0 Axes>